In [1]:
import pandas as pd
import sqlalchemy as sql
import json

In [2]:
# db connection
# username = "FPSINPUT"
# password = "SkyOra#DwhSch#2021"
# host = "SGDWHDEV.ad.skynet"
# port = "1521"
# sid = "SGDWHDEV"

# engine = sql.create_engine(f"oracle+oracledb://{username}:{password}@{host}:{port}/{sid}")
from sql_connection import engine
engine.connect()




In [3]:
query_pins = """
select
    prcdname,
    prcdversion,
    pinsinstnum,
    callprcdid,
    executerecpid
from plld.pins@sgodsprd
"""

df_pins = pd.read_sql(query_pins, con=engine)
print(df_pins.head())

  prcdname prcdversion pinsinstnum callprcdid executerecpid
0    #REF!         .01     001.000                         
1    #REF!         .01     002.000      #REF!              
2    #REF!         .01     003.000                         
3        0         .01     001.000                         
4        0         .01     002.000          -              


In [4]:
query_prcd = """
select 
    prcdname,
    prcdversion,
    activeflag
from plld.prcd@sgodsprd
"""

df_prcd = pd.read_sql(query_prcd, con=engine)
print(df_prcd.head())

              prcdname prcdversion activeflag
0                #REF!         .01           
1                    0         .01           
2         1KD722MHW81A         .01          A
3  1ST-PIEZO-FORMATION         .01          A
4         1ST-TRIMMING         .01           


In [5]:

# Assuming connection to the database is already established as 'connection'
# Replace 'connection' with your actual database connection object

# First SQL Query Implementation in Python
query_hist = """
SELECT 
    Partid,
    PARTNAME,
    partversion,
    technology,
    curprcdid,
    curprcdname,
    curprcdversion,
    timestamptime 
FROM (
    SELECT 
        Partid,
        PARTNAME,
        partversion,
        substr(lotid,1,7) as baselot,
        parentid,
        lotid,
        timestamptime,
        curprcdid,
        curprcdname,
        curprcdversion,
        recpid,
        lottype,
        priority,
        comclass,
        eqptype,
        eqpid,
        location,
        queuetime,
        trackintime,
        startrecptime,
        endrecptime,
        trackouttime,
        case
            when prodarea = 'SINGAPORE' then 'SAW'
            when prodarea = 'SINGAPORE3' then 'BAW'
            else 'UNKNOWN'
        end as technology,
        case
            when endrecptime is not null and startrecptime is not null then trunc((endrecptime - startrecptime)*86400)
            when trackouttime is not null and trackintime is not null then trunc((trackouttime - trackintime)*86400)
            else 0
        end as recipe_time_second,
        ROW_NUMBER() OVER (PARTITION BY partid ORDER BY timestamptime DESC) AS row_num
    FROM plld.hist@sgodsprd
    WHERE 
        prodarea in ('SINGAPORE','SINGAPORE3')
        and lottype in ('E', 'P')
        AND TIMESTAMPTIME BETWEEN TO_TIMESTAMP('01-JAN-23 00.00.00', 'DD-MON-YY HH24.MI.SS') AND SYSTIMESTAMP
        and curprcdname = partname
) subquery
WHERE row_num = 1
order by timestamptime DESC
"""
df_hist = pd.read_sql(query_hist, con=engine)
print(df_hist.head())

            partid      partname partversion technology        curprcdid  \
0  SA106BA715MD.01  SA106BA715MD         .01        SAW  SA106BA715MD.22   
1  BA175NA2147D.01  BA175NA2147D         .01        SAW  BA175NA2147D.10   
2  BA173NA1842D.01  BA173NA1842D         .01        SAW  BA173NA1842D.11   
3  EE077NA869MD.01  EE077NA869MD         .01        SAW  EE077NA869MD.07   
4  JC005RB680MD.01  JC005RB680MD         .01        SAW  JC005RB680MD.17   

    curprcdname curprcdversion       timestamptime  
0  SA106BA715MD            .22 2025-02-14 11:26:55  
1  BA175NA2147D            .10 2025-02-14 11:00:59  
2  BA173NA1842D            .11 2025-02-14 10:53:18  
3  EE077NA869MD            .07 2025-02-14 10:48:54  
4  JC005RB680MD            .17 2025-02-14 10:47:33  


In [6]:
# length of df_hist
print(len(df_hist))

1512


In [7]:
def get_active_prcd(df_prcd, prcd_name):
    """Fetch the active row (ActiveFlag = 'A') from df_prcd based on prcd_name."""
    active_row = df_prcd[(df_prcd['prcdname'] == prcd_name) & (df_prcd['activeflag'] == 'A')]
    return active_row.iloc[0]['prcdversion'] if not active_row.empty else None

In [8]:
def fetch_composite_results(df_pins, prcd_name, prcd_version):
    """Fetch rows from df_pins ordered by pinsinstnum."""
    results = df_pins[(df_pins['prcdname'] == prcd_name) & (df_pins['prcdversion'] == prcd_version)]
    return results.sort_values(by='pinsinstnum').to_dict('records')

In [9]:
def recursive_fetch(df_pins, df_prcd, callprcd_id, results):
    """ function to fetch recipe based on callprcdid."""
    active_version = get_active_prcd(df_prcd, callprcd_id)
    if active_version is not None:
        fetched_rows = fetch_composite_results(df_pins, callprcd_id, active_version)
        # returned list of executerecpid
        fetched_recipes = [row['executerecpid'] for row in fetched_rows if row['executerecpid'].strip()]
        if fetched_recipes:
            results.extend(fetched_recipes)

In [10]:

def process_ids(df, df_pins, df_prcd):
    """Main function to process all IDs and compile results."""
    final_results = {}

    # for all partids
    for _, row in df.iterrows():
        partid = row['partid']
        partname = row['partname']
        partversion = row['partversion']
        technology = row['technology']
        curprcdid = row['curprcdid']
        curprcdname = row['curprcdname']
        curprcdversion = row['curprcdversion']

        # stage partname = prcdname
        # pins_result is the row with material name
        pins_result = df_pins.loc[(df_pins['prcdname'] == curprcdname) & (df_pins['prcdversion'] == curprcdversion) & df_pins['callprcdid'].str.strip().astype(bool)]
        # print(pins_result)
        if pins_result.empty:
            print(f"No material name found for partid: {partid}")

        # stage prcdname = material name
        material_id = pins_result.iloc[0]['callprcdid']
        # active material version
        active_version= get_active_prcd(df_prcd, material_id)
        if active_version is None:
            print(f"No active version found for material_id: {material_id}")

        list_of_procedure = fetch_composite_results(df_pins, material_id, active_version)
        fetched_results = []
        for row in list_of_procedure:
            callprcd_id = row['callprcdid'].strip()
            if pd.notna(callprcd_id):
                recursive_fetch(df_pins, df_prcd, callprcd_id, fetched_results)


        final_results[partid] = {
            "partid": partid,
            "partname": partname,
            "partversion": partversion,
            "technology": technology,
            "curprcdid": callprcd_id,
            "recipe": fetched_results
        }

    return final_results

# Process and compile all IDs
compiled_results = process_ids(df_hist, df_pins, df_prcd)

# Write results to a JSON file
with open('PartID_Recipe.json', 'w') as json_file:
    json.dump(compiled_results, json_file, indent=4)

print("Results have been saved to PartID_Recipe.json")



Results have been saved to PartID_Recipe.json
